In [11]:
import phu_yolov5
import cv2
import os
import torch
import numpy as np
import pandas as pd

IOU_THRESHORD = 0.5
CONFIDENCE_THRESHORD = 0.7

In [12]:
#test img dir
test_img_dir = "test_data/images/"
test_label_dir = "test_data/labels/"


imgs_name = os.listdir( test_img_dir )
img_name = imgs_name[1]

#read img
# img_path = test_img_dir + img_name
# img = cv2.imread(img_path)
# cv2.imshow("adsf",img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [13]:
#load model
model_path = "best.pt"
model = torch.hub.load( "./yolov5", "custom", model_path, source="local")
model.conf = CONFIDENCE_THRESHORD

YOLOv5  v6.1-243-g7cef03d Python-3.9.12 torch-1.8.2+cu111 CUDA:0 (NVIDIA GeForce MX130, 2048MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [14]:
# get detection
def get_predict_boxes(img_name):
	img = cv2.imread( test_img_dir+img_name)
	predict_boxes = phu_yolov5.get_detection( img, model )
	return predict_boxes 

In [15]:
def load_truth_boxes( img_name ):
	# load truth boxes
	img_file = open(test_label_dir + img_name[:-3] + "txt")
	img_file_content = img_file.readlines()
	img_file_content_lines = [line.split()[1:] for line in img_file_content]
	img_file_content_lines = np.array(img_file_content_lines)

	img_file.close()

	# format of 1 box
	# xcen ycen w h
	truth_boxes = img_file_content_lines.astype(float)
	truth_boxes

	# convert to Phu's detection's format
	### get real size
	img = cv2.imread(test_img_dir + img_name)
	real_h,real_w,d = img.shape
	## convert to pixel
	converted_truth_boxes = []
	for box in truth_boxes:
		xcen = box[0] *real_w
		ycen = box[1] *real_h
		box_width = box[2] *real_w
		box_height = box[3] *real_h

		xmin = xcen - box_width/2
		ymin = ycen - box_height/2
		xmax = xcen + box_width/2
		ymax = ycen + box_height/2
		converted_truth_boxes += [[xmin,ymin,xmax,ymax]]

	return converted_truth_boxes

In [16]:
def get_boxes(img_name):
	predict_boxes = get_predict_boxes(img_name)
	truth_boxes = load_truth_boxes(img_name)
	return predict_boxes, truth_boxes

def get_Tu_requied(img_name):
	predict_boxes,truth_boxes = get_boxes(img_name)
	result = []
	
	for i in range(len(predict_boxes)):
		line = [img_name, i]
		maxIOU = 0
		for tru in truth_boxes:
			curIOU = phu_yolov5.IOU(tru, predict_boxes[i][:-2])
			maxIOU = max(curIOU,maxIOU)
		line += [maxIOU, predict_boxes[i][-2]]
		# add TP of FP
		if maxIOU > IOU_THRESHORD:
			line += ["TP"]
		else:
			line += ["FP"]

		result.append(line)

	return result
	

In [17]:
imgs_name = os.listdir( test_img_dir )
matrix_2d_result = []
for img_name in imgs_name:
	matrix_2d_result += get_Tu_requied(img_name)

df_result = pd.DataFrame(matrix_2d_result,columns=["Image_Name","No","IOU","Confidence","True or False"])

In [19]:
df_result.to_csv("result.csv")

In [20]:
## def get_TP_FP_FN():

IOU_Threshord = IOU_THRESHORD

TP = len(df_result[df_result.IOU > IOU_Threshord]) 
FP = len(df_result[df_result.IOU <= IOU_Threshord])
FN = 0

for img_name in imgs_name:
	predict_boxes, truth_boxes = get_boxes(img_name)
	for tru in truth_boxes:
		maxIOU = 0
		for pre in predict_boxes:
			curIOU = phu_yolov5.IOU(tru,pre[:-2])
			maxIOU = max(curIOU,maxIOU)
		
		if maxIOU <= IOU_Threshord : 
			FN += 1

TP, FP, FN


(1062, 51, 88)

In [21]:
# prepare for AP
truth_dir =  "mAP/input/ground-truth/"
detect_dir = "mAP/input/detection-results/"
for img_name in imgs_name:
	print( img_name + "...")

	detect_boxes, truth_boxes = get_boxes(img_name)
	label_file_name = img_name[:-3]+"txt"

	# truth content
	truth_content = []
	for box in truth_boxes:
		line = " ".join(["biker"] + [str(int(x)) for x in box])
		truth_content += [line]

	detect_content = []
	for box in detect_boxes:
		line = " ".join(["biker"] + [str(box[-2])] + [str(int(x)) for x in box[:-2]])
		detect_content += [line]

	# write to truth
	truth_file = open(truth_dir + label_file_name, "w+")
	truth_file.write("\n".join(truth_content))
	truth_file.close()

	detect_file = open(detect_dir + label_file_name, "w+")
	detect_file.write("\n".join(detect_content))
	detect_file.close()




consau (129).jpg...
consau (132).jpg...
consau (153).jpg...
consau (16).jpg...
consau (168).jpg...
consau (175).jpg...
consau (178).jpg...
consau (186).jpg...
consau (187).jpg...
consau (188).jpg...
consau (190).jpg...
consau (196).jpg...
consau (198).jpg...
consau (200).jpg...
consau (203).jpg...
consau (226).jpg...
consau (241).jpg...
consau (34).jpg...
consau (36).jpg...
consau (40).jpg...
consau (79).jpg...
consau (96).jpg...
cropped_thai_104.jpg...
cropped_thai_26.jpg...
cropped_thai_35.jpg...
cropped_thai_36.jpg...
cropped_thai_52.jpg...
cropped_thai_54.jpg...
cropped_thai_61.jpg...
cropped_thai_69.jpg...
cropped_thai_71.jpg...
cropped_thai_81.jpg...
cropped_thai_83.jpg...
dendo (18).jpg...
dendo (25).jpg...
dendo (27).jpg...
dendo (37).jpg...
dendo (38).jpg...
img_UIT_104.jpg...
img_UIT_111.jpg...
img_UIT_120.jpg...
img_UIT_133.jpg...
img_UIT_134.jpg...
img_UIT_138.jpg...
img_UIT_155.jpg...
img_UIT_172.jpg...
img_UIT_184.jpg...
img_UIT_185.jpg...
img_UIT_191.jpg...
img_UIT_2.jpg